# Clean Event Data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
DIR = "/workspace/nflbigdatabowl2023"
sys.path.append(DIR)

In [3]:
import pandas as pd

from nflpocketarea2023.pipeline.tasks.events import clean_event_data
from nflpocketarea2023.pipeline.tasks.dataframes import read_tracking_data
from nflpocketarea2023.pipeline.flows.main import main_flow

In [4]:
df_tracking_all = read_tracking_data(f"{DIR}/data/raw/week", weeks=8)

In [5]:
for event in sorted(df_tracking_all["event"].unique()):
    print(event)

None
autoevent_ballsnap
autoevent_passforward
autoevent_passinterrupted
ball_snap
dropped_pass
first_contact
fumble
fumble_offense_recovered
handoff
huddle_break_offense
lateral
line_set
man_in_motion
out_of_bounds
pass_arrived
pass_forward
pass_outcome_caught
pass_outcome_incomplete
pass_tipped
penalty_flag
play_action
qb_sack
qb_strip_sack
run
shift
tackle
